In [2]:
#importing dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import re
import nltk
from nltk.corpus import stopwords
import string
import json
from time import time
import pickle
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from keras.layers.merge import add

In [3]:
# tf.compat.v1.enable_eager_execution()
# K.clear_session()
# tf.compat.v1.reset_default_graph()
#  import tensorflow.compat.v1 as tf
#  tf.disable_v2_behavior()
#  x = tf.placeholder(shape=[None, 2], dtype=tf.float32)

In [4]:
model = load_model('./model_weights/model_9.h5')

In [5]:
modell = ResNet50(weights="imagenet",input_shape=(224,224,3))

In [6]:
modeln = Model(modell.input,modell.layers[-2].output)

In [7]:
with open("./w2i.pkl","rb") as f:
    word_to_idx=pickle.load(f)
with open("./i2w.pkl","rb") as g:
    idx_to_word=pickle.load(g)

In [8]:
def preprocess_img(img):
    img = image.load_img(img,target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    # Normalisation
    img = preprocess_input(img)
    return img

In [9]:
def encode_image(img):
    img = preprocess_img(img)
    feature_vector = modeln.predict(img)
    feature_vector = feature_vector.reshape((1,feature_vector.shape[1]))
    #print(feature_vector.shape)
    return feature_vector

In [10]:
max_len=35
def predict_caption(photo):
    in_text = "startseq"
    for i in range(max_len):
        sequence = [word_to_idx[w] for w in in_text.split() if w in word_to_idx]
        sequence = pad_sequences([sequence],maxlen=max_len,padding='post')
        ypred = model.predict([photo,sequence])
        ypred = ypred.argmax() #WOrd with max prob always - Greedy Sampling
        word = idx_to_word[ypred]
        in_text += (' ' + word)        
        if word == "endseq":
            break   
    final_caption = in_text.split()[1:-1]
    final_caption = ' '.join(final_caption)
    return final_caption

In [11]:
def captionis(img):
    enc=encode_image(img)
    cap=predict_caption(enc)
    return cap

In [13]:
enc=encode_image("ashi.jpg")

In [14]:
predict_caption(enc)

'man in red shirt is standing on the edge of the mountain'

In [35]:
#